# File Notebook untuk Machine Learning Competition

In [1]:
import numpy as np
import pandas as pd

#load data csv
df1 = pd.read_csv('https://drive.google.com/uc?id=14yUYJH-tz-tj4oOiGn7Bkbjh0cz-Y0OX')
df2 = pd.read_csv('https://drive.google.com/uc?id=1-E8qd6nEIqHK774rUVaWhEDBRuwA113w')

## Tahap pertama adalah memanipulasi data agar ia dapat diproses pada model machine learning selanjutnya

In [2]:
#Sebelum diload, saya sudah periksa data csv, dan ia mengandung data kosong dengan value string '?'
#Replace data kosong dengan NaN
df1 = df1.replace('?', np.nan)
df2 = df2.replace('?', np.nan)

In [3]:
#Isi data NaN dengan method frontfill
df1 = df1.fillna(method='ffill')
df2 = df2.fillna(method='ffill')

In [4]:
#Encode data untuk kolom berisi data string
df1 = pd.get_dummies(df1, columns=['Kelas Pekerja', 'Status Perkawinan',
             'Pekerjaan'])
df2 = pd.get_dummies(df2, columns=['Kelas Pekerja', 'Status Perkawinan',
             'Pekerjaan'])

In [5]:
#Replace kolom pendidikan dengan data ordeal
col1 = {'SD':1,'1st-4th':2, '5th-6th':3, '7th-8th':4, 
        '9th':5, '10th':6, '11th':7, '12th':8, 'SMA':9, 
        'Pendidikan Tinggi':10, 'D4':11, 'D3':12, 'Sarjana':13, 
        'Master':14, 'Sekolah Professional':15, 'Doktor':16 }
df1['Pendidikan'] = df1['Pendidikan'].replace(col1)
df2['Pendidikan'] = df2['Pendidikan'].replace(col1)

In [6]:
#replace kolom jenis kelamin dengan data ordeal
col2 = {'Laki2':1, 'Perempuan':2}
df1['Jenis Kelamin'] = df1['Jenis Kelamin'].replace(col2)
df2['Jenis Kelamin'] = df2['Jenis Kelamin'].replace(col2)

In [7]:
#replace data label
col3 = {'<=7jt':0, '>7jt':1}
df1['Gaji'] = df1['Gaji'].replace(col3)

## Untuk percobaan model, saya akan menggunakan dua model, yaitu KNN dan Decision Tree Regressor

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [9]:
#Pemilihan data train, dan data test
X_train = df1.drop('Gaji', axis=1)
y_train = df1['Gaji']
y_train = y_train

X_test = df2

### Model Pertama: KNN

In [ ]:
#Percobaan pertama menggunakan model KNN
#Ujicoba tuning hyperparameter KNN
model = KNeighborsClassifier()
param_grids = {
    'n_neighbors':np.arange(10,40), 
    'weights': ['uniform','distance'], 
}
rscv = RandomizedSearchCV(model, param_distributions=param_grids, n_iter=25, scoring='roc_auc', cv=5)
rscv.fit(X_train,y_train)

In [ ]:
#dapatkan hyperparameter terbaik
rscv.best_params_

In [ ]:
#dapatkan score terbaik
rscv.best_score_

In [ ]:
#Gunakan hyperparameter terbaik setelah tuning pada model KNN
model = KNeighborsClassifier(n_neighbors=24, weights='distance')
model.fit(X_train, y_train)

In [ ]:
#prediksi data test model KNN
y_pred = model.predict(X_test)

In [ ]:
#Ubah data test menjadi data Frame
cek = pd.DataFrame(y_pred, columns=['Gaji'])

In [ ]:
#Simpan data test pada file csv
#Input index di sini masih dimulai dari nol
#Untuk urutan index dimulai dari 35994 diinput manual pada saat membuka file csv
cek.to_csv('prediction1.csv')

### Model Kedua: Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
#Percobaan kedua menggunakan model Decision Tree Regressor
#Ujicoba tuning hyperparameter Decision Tree Regressor
#Untuk menghemat waktu, saya lakukan per-penilaian per-parameter. Selalu didapat untuk max depth nilai 8 adalah terbaik
#Begitu pula nilai hyperparameter selanjutnya
model2 = DecisionTreeRegressor()
param2_grids = {
    'max_depth':np.arange(1,10),
    'min_samples_split':[5],
    'min_samples_leaf':np.arange(1,5)
}
gscv = GridSearchCV(model2, param_grid=param2_grids, scoring='roc_auc', cv=10)
gscv.fit(X_train,y_train)

In [ ]:
#dapatkan hyperparameter terbaik
gscv.best_params_

In [ ]:
#dapatkan score terbaik
gscv.best_score_

In [ ]:
#Gunakan hyperparameter terbaik setelah tuning pada model Decision Tree Regressor
model2 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=3, min_samples_split=5)
model2.fit(X_train, y_train)

In [ ]:
#prediksi data test model Decision Tree Regressor
y_pred = model2.predict(X_test)

In [ ]:
#Ubah data test menjadi data Frame
cek = pd.DataFrame(y_pred, columns=['Gaji'])

In [ ]:
#Simpan data test pada file csv
#Input index di sini masih dimulai dari nol
#Untuk urutan index dimulai dari 35994 diinput manual pada saat membuka file csv
cek.to_csv('prediction2.csv')

## Sejauh ini pada percobaan saya, model Decision Tree Regressor menghasilkan scoring lebih besar dibandingkan dengan model KNN. Dapat dikatakan bahwa, model Decision Tree Regressor adalah model yang cocok untuk data ini.